<a href="https://colab.research.google.com/github/KTH-EXPECA/examples/blob/main/openairinterface/gnbcoreseparated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Authentication and Dependencies

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [12]:
import os, re
from getpass import getpass

with open('/content/demo_project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········


Install required packages and dependencies. Ignore the warnings.

In [13]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


Import packages

In [14]:
import json, time
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, show_reservation_byname, restart_sdr, make_sdr_ni, make_sdr_mango, sdr_tools, get_available_publicips, get_segment_ids, get_radio_interfaces, get_worker_interfaces

# Reserve resources

In the next cell, we reserve 1 SDR and 1 worker.

In [4]:
# Check the SDR's health and the status of its ports, both ports are supposed to be up, otherwise contact support
sdr_name = "sdr-10"
sdr_status = get_radio_interfaces(sdr_name)
logger.info(f"{json.dumps(sdr_status, indent=4)}")
for port in sdr_status.keys():
  if sdr_status[port]['linkstate'] == 'Down':
    logger.warning(f"port {port} on {sdr_name} is down.")
  if sdr_status[port]['linkstate'] == 'Up':
    logger.success(f"port {port} on {sdr_name} is up.")

2024-07-16 15:20:09.187 | INFO     | __main__:<cell line: 4>:4 - {
    "sdr_10_mango": {
        "backpressure": "Disabled",
        "duplex": "Full",
        "flowctrl": "Off",
        "linkstate": "Up",
        "mdixmode": "On",
        "neg": "Enabled",
        "port": "te1/0/2",
        "segment_id": 119,
        "speed": "1000",
        "stitches": {
            "119": []
        },
        "type": "10G-Copper"
    },
    "sdr_10_ni": {
        "backpressure": "Disabled",
        "duplex": "Full",
        "flowctrl": "Off",
        "linkstate": "Up",
        "mdixmode": "Off",
        "neg": "Disabled",
        "port": "te2/0/16",
        "segment_id": 120,
        "speed": "10000",
        "stitches": {
            "120": []
        },
        "type": "10G-Fiber"
    }
}
2024-07-16 15:20:09.189 | SUCCESS  | __main__:<cell line: 5>:9 - port sdr_10_mango on sdr-10 is up.
2024-07-16 15:20:09.192 | SUCCESS  | __main__:<cell line: 5>:9 - port sdr_10_ni on sdr-10 is up.


In [15]:
# Reserve the SDR
segment_ids = get_segment_ids(sdr_name)

# reserve RJ45 port
rj45_lease = show_reservation_byname(sdr_name + "-rj45-lease")
if not rj45_lease:
    rj45_lease = reserve(
        { "type":"network", "name": sdr_name+"-rj45", "net_name": sdr_name+"-rj45", "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

# reserve SFP port
sfp_lease = show_reservation_byname(sdr_name + "-sfp-lease")
if not sfp_lease:
    sfp_lease = reserve(
        { "type":"network", "name": sdr_name+"-sfp", "net_name": sdr_name+"-sfp", "segment_id": segment_ids['sfp'], "duration": { "days":7, "hours":0 } }
    )

worker_name = 'worker-07'
# reserve worker
worker_lease = show_reservation_byname(worker_name+"-lease")
if not worker_lease:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
    )
worker_reservation_id = worker_lease["reservations"][0]["id"]


leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

2024-07-16 18:08:43.885 | INFO     | chi.expeca:reserve:243 - reserving worker-07
2024-07-16 18:08:46.146 | INFO     | chi.expeca:wait_until_lease_status:138 - waiting 120 seconds for worker-07-lease with id 7cbadf29-69a2-400a-bf1f-782e6dc702b3 to become "ACTIVE"
2024-07-16 18:08:51.300 | INFO     | chi.expeca:wait_until_lease_status:145 - lease worker-07-lease with id 7cbadf29-69a2-400a-bf1f-782e6dc702b3 is PENDING.
2024-07-16 18:08:56.474 | INFO     | chi.expeca:wait_until_lease_status:145 - lease worker-07-lease with id 7cbadf29-69a2-400a-bf1f-782e6dc702b3 is PENDING.
2024-07-16 18:09:01.625 | INFO     | chi.expeca:wait_until_lease_status:145 - lease worker-07-lease with id 7cbadf29-69a2-400a-bf1f-782e6dc702b3 is PENDING.
2024-07-16 18:09:06.780 | INFO     | chi.expeca:wait_until_lease_status:145 - lease worker-07-lease with id 7cbadf29-69a2-400a-bf1f-782e6dc702b3 is PENDING.
2024-07-16 18:09:11.938 | INFO     | chi.expeca:wait_until_lease_status:145 - lease worker-07-lease with id 

[
    {
        "name": "sdr-10-rj45-lease",
        "id": "18b7fbc7-1482-4e2f-9f13-632ee4500e1e",
        "reservation_id": "cc53d8fb-9b3e-4e65-9778-8b53dd6ab6a1",
        "status": "ACTIVE",
        "end_date": "2024-07-22T19:07:00.000000"
    },
    {
        "name": "adv-06-lease",
        "id": "450200ac-6df7-4d2b-a217-d4aa006d5d97",
        "reservation_id": "233590da-2e3c-4be0-86be-204f891ba820",
        "status": "ACTIVE",
        "end_date": "2024-07-23T16:08:00.000000"
    },
    {
        "name": "worker-07-lease",
        "id": "7cbadf29-69a2-400a-bf1f-782e6dc702b3",
        "reservation_id": "6a3ae604-c8f3-4243-af4c-db1999dccb19",
        "status": "ACTIVE",
        "end_date": "2024-07-23T18:08:00.000000"
    },
    {
        "name": "sdr-10-sfp-lease",
        "id": "ad69a46f-013b-4f69-a486-a2ad3dbe06aa",
        "reservation_id": "ed41686e-061d-4fc3-b754-987d3452ced3",
        "status": "ACTIVE",
        "end_date": "2024-07-22T19:08:00.000000"
    },
    {
        "nam

# Configure the SDRs

## Change the sdrs' firmwares to Mango or NI

In [ ]:
# change sdr design to ni using the reserved worker
sdrnet = chi.network.get_network(sdr_name+"-rj45-net")
make_sdr_ni(sdr_name,sdrnet['id'],worker_reservation_id,"ens1")

# wait 10 seconds
#time.sleep(10)

# change sdr design to mango using the reserved worker
#sdrnet = chi.network.get_network(sdr_name+"-rj45-net")
#make_sdr_mango(sdr_name,sdrnet['id'],worker_reservation_id,"ens1")

2023-09-01 08:04:04.639 | SUCCESS  | chi.expeca:make_sdr_mango:299 - created make-sdr-mango container.
2023-09-01 08:04:04.641 | INFO     | chi.expeca:make_sdr_mango:301 - waiting 2 minutes for the sdr-09 to change design.
2023-09-01 08:05:06.410 | SUCCESS  | chi.expeca:make_sdr_mango:315 - SERVICE=change_design env variable is set
running change_design...
You have chosen sdr-09 and design mango
sdr-09 is reachable: {'mango': {'ip': '10.30.1.17', 'port': '22', 'tenant-port': 'te1/0/26', 'up': False}, 'ni': {'ip': '10.30.1.18', 'port': '22', 'tenant-port': 'te4/0/13', 'up': True}}
SSHing to 10.30.1.18, username:root, password: 
running command on 10.30.1.18: 
cp /uboot/mango_bootbin/BOOT.bin /uboot/ ; /sbin/reboot > /dev/null 2>&1 &
command stdout: 

command sdterr: 

Waiting 200 seconds for the new desgin to load
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
sdr-09 design has be

## Reboot the SDRs

In [ ]:
# reset sdr using the reserved worker
sdrnet = chi.network.get_network(sdr_name+"-rj45-net")
restart_sdr(sdr_name,sdrnet['id'],worker_reservation_id,"ens1")


2023-09-01 10:10:24.442 | SUCCESS  | chi.expeca:restart_sdr:255 - created reboot-sdr container.
2023-09-01 10:10:24.445 | INFO     | chi.expeca:restart_sdr:257 - waiting 2 minutes for the sdr-09 to reboot.
2023-09-01 10:11:15.129 | SUCCESS  | chi.expeca:restart_sdr:267 - SERVICE=reboot env variable is set
running reboot...
You have chosen sdr-09
sdr-09 is reachable: {'mango': {'ip': '10.30.1.17', 'port': '22', 'tenant-port': 'te1/0/26', 'up': True}, 'ni': {'ip': '10.30.1.18', 'port': '22', 'tenant-port': 'te4/0/13'}}
SSHing to 10.30.1.17, username:root, password: root
running command on 10.30.1.17: 
/sbin/reboot > /dev/null 2>&1 &
Waiting 200 seconds for the sdr to load
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
sdr-09 mango is up again.

2023-09-01 10:11:15.612 | INFO     | chi.expeca:wait_until_container_removed:30 - waiting 30 seconds for reboot-sdr container to be removed
2023-09-01 10:11:20.836 | INFO

# Run A Public Container to Test the SDR

Run a public container to test the SDR via its SFP port

In [11]:
# advantech router reservation
adv_name = "adv-06"
segment_ids = get_segment_ids(adv_name)
adv_lease = show_reservation_byname(name + "-lease")
if not adv_lease:
    adv_lease = reserve(
        { "type":"network", "name": adv_name, "net_name": adv_name, "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

2024-07-16 16:08:42.721 | INFO     | chi.expeca:reserve:243 - reserving adv-06
2024-07-16 16:08:45.291 | INFO     | chi.expeca:wait_until_lease_status:138 - waiting 120 seconds for adv-06-lease with id 450200ac-6df7-4d2b-a217-d4aa006d5d97 to become "ACTIVE"
2024-07-16 16:08:50.448 | INFO     | chi.expeca:wait_until_lease_status:145 - lease adv-06-lease with id 450200ac-6df7-4d2b-a217-d4aa006d5d97 is PENDING.
2024-07-16 16:08:55.602 | INFO     | chi.expeca:wait_until_lease_status:145 - lease adv-06-lease with id 450200ac-6df7-4d2b-a217-d4aa006d5d97 is PENDING.
2024-07-16 16:09:00.756 | INFO     | chi.expeca:wait_until_lease_status:145 - lease adv-06-lease with id 450200ac-6df7-4d2b-a217-d4aa006d5d97 is PENDING.
2024-07-16 16:09:05.938 | INFO     | chi.expeca:wait_until_lease_status:145 - lease adv-06-lease with id 450200ac-6df7-4d2b-a217-d4aa006d5d97 is STARTING.
2024-07-16 16:09:11.098 | INFO     | chi.expeca:wait_until_lease_status:145 - lease adv-06-lease with id 450200ac-6df7-4d2b-a

In [18]:
# Create core network
corenet = chi.network.create_network("oai-core-net")
chi.network.create_subnet("oai-core-subnet",corenet["id"],"192.168.70.128/26",gateway_ip="192.168.70.190",enable_dhcp=False)
logger.success("oai-core-net is created.")

2024-07-16 19:09:40.115 | SUCCESS  | __main__:<cell line: 4>:4 - oai-core-net is created.


In [17]:
# Assign workers
core_worker_name = 'worker-07'
gnb_worker_name = 'worker-01'

core_worker_reservation_id = None
gnb_worker_reservation_id = None
leaseslist = list_reservations(brief=True)
for lease in leaseslist:
  if core_worker_name in lease['name']:
    core_worker_reservation_id = lease['reservation_id']
    logger.success(f"core {core_worker_name} reservation found")
  if gnb_worker_name in lease['name']:
    gnb_worker_reservation_id = lease['reservation_id']
    logger.success(f"gnb {gnb_worker_name} reservation found")

2024-07-16 19:09:16.202 | SUCCESS  | __main__:<cell line: 8>:11 - core worker-07 reservation found
2024-07-16 19:09:16.204 | SUCCESS  | __main__:<cell line: 8>:14 - gnb worker-01 reservation found


In [19]:
# Run corenetwork container
worker_name = core_worker_name
worker_reservation_id = core_worker_reservation_id

# check public IPs and select one
available_pub_ips = get_available_publicips()
if len(available_pub_ips) == 0:
  raise Exception("There is no available public IPs to reserve.")
pub_ip = available_pub_ips[0]
logger.info(f"Available public ips: {available_pub_ips}.")
logger.info(f"We choose {pub_ip} for this container.")

# check available interfaces on the worker
interfaces = list(get_worker_interfaces(worker_name).values())[0]
available_ifs = []
for interface in interfaces.keys():
  if len(interfaces[interface]['connections']) == 0:
    available_ifs.append(interface)
logger.info(f"Available interfaces on {worker_name}: {available_ifs}")
# we need 2 interfaces
# one for public access and one for core network
if len(available_ifs) < 2:
  logger.info(f"{interfaces}")
  raise Exception(f"did not find 2 free interfaces on worker {worker_name}")

# run the gnb container
publicnet = chi.network.get_network("serverpublic")
oaicorenet = chi.network.get_network("oai-core-net")
container_name = "oai-core-host"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/sshd-dind",
    reservation_id = worker_reservation_id,
    environment = {
        "DNS_IP":"8.8.8.8",
        "GATEWAY_IP":"130.237.11.97",
        "PASS":"expeca"
    },
    mounts = [],
    nets = [
        { "network" : publicnet['id'] },
        { "network" : oaicorenet['id'] },
    ],
    labels = {
        "networks.1.interface":available_ifs[0],
        "networks.1.ip":pub_ip+"/27",
        "networks.1.gateway":"130.237.11.97",
        "networks.2.interface":available_ifs[1],
        "networks.2.ip":"192.168.70.190/26",
        "capabilities.privileged":"true",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container, reachable at {pub_ip}.")

2024-07-16 19:10:11.866 | INFO     | __main__:<cell line: 10>:10 - Available public ips: ['130.237.11.114', '130.237.11.115', '130.237.11.116', '130.237.11.117', '130.237.11.119', '130.237.11.120', '130.237.11.121', '130.237.11.123'].
2024-07-16 19:10:11.868 | INFO     | __main__:<cell line: 11>:11 - We choose 130.237.11.114 for this container.
2024-07-16 19:10:25.281 | INFO     | __main__:<cell line: 19>:19 - Available interfaces on worker-07: ['eno12409', 'eno12419', 'eno12429', 'ens1']
2024-07-16 19:11:01.810 | SUCCESS  | __main__:<cell line: 54>:54 - created oai-core-host container, reachable at 130.237.11.114.


In [20]:
# Run gnb container
worker_name = gnb_worker_name
worker_reservation_id = gnb_worker_reservation_id

# check public IPs and select one
available_pub_ips = get_available_publicips()
if len(available_pub_ips) == 0:
  raise Exception("There is no available public IPs to reserve.")
pub_ip = available_pub_ips[0]
logger.info(f"Available public ips: {available_pub_ips}.")
logger.info(f"We choose {pub_ip} for this container.")

# check available interfaces on the worker
interfaces = list(get_worker_interfaces(worker_name).values())[0]
available_ifs = []
for interface in interfaces.keys():
  if len(interfaces[interface]['connections']) == 0:
    available_ifs.append(interface)
logger.info(f"Available interfaces on {worker_name}: {available_ifs}")
# we need one 10Gbps interface for the SDR,
# and one port with any speed for the public interface
port_10g = None
port1_any = None
port2_any = None
for interface in available_ifs:
  if (port_10g is None) and (interfaces[interface]['speed'] == '10000'):
    port_10g = interface
    continue
  if port1_any is None:
    port1_any = interface
    continue
  if port2_any is None:
    port2_any = interface
    continue
if (port_10g is None) or (port1_any is None) or (port2_any is None):
  logger.info(f"{json.dumps(interfaces, indent=4)}")
  raise Exception(f"Did not find proper interfaces on {worker_name}")
else:
  logger.success(f"we choose {port_10g} for the SDR, {port1_any} for the public access, and {port2_any} for the core network.")

# run the gnb container
sdrsfpnet = chi.network.get_network(sdr_name+"-sfp-net")
publicnet = chi.network.get_network("serverpublic")
oaicorenet = chi.network.get_network("oai-core-net")
container_name = "gnb-sdr-host"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/sshd-image",
    reservation_id = worker_reservation_id,
    environment = {
        "DNS_IP":"8.8.8.8",
        "GATEWAY_IP":"130.237.11.97",
        "PASS":"expeca"
    },
    mounts = [],
    nets = [
        { "network" : publicnet['id'] },
        { "network" : oaicorenet['id'] },
        { "network" : sdrsfpnet['id'] },
    ],
    labels = {
        "networks.1.interface":port1_any,
        "networks.1.ip":pub_ip+"/27",
        "networks.1.gateway":"130.237.11.97",
        "networks.2.interface":port2_any,
        "networks.2.ip":"192.168.70.129/26",
        "capabilities.privileged":"true",
        "networks.3.interface":port_10g,
        "networks.3.ip":"10.30.10.120/24",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container, reachable at {pub_ip}.")


2024-07-16 19:14:26.250 | INFO     | __main__:<cell line: 10>:10 - Available public ips: ['130.237.11.115', '130.237.11.116', '130.237.11.117', '130.237.11.119', '130.237.11.120', '130.237.11.121', '130.237.11.123'].
2024-07-16 19:14:26.252 | INFO     | __main__:<cell line: 11>:11 - We choose 130.237.11.115 for this container.
2024-07-16 19:14:39.133 | INFO     | __main__:<cell line: 19>:19 - Available interfaces on worker-01: ['eno12399', 'eno12409', 'eno12419', 'eno12429', 'ens1f1']
2024-07-16 19:14:39.135 | SUCCESS  | __main__:<cell line: 35>:39 - we choose ens1f1 for the SDR, eno12399 for the public access, and eno12409 for the core network.
2024-07-16 19:15:17.352 | SUCCESS  | __main__:<cell line: 73>:73 - created gnb-sdr-host container, reachable at 130.237.11.115.



The following must be set in the docker compose as for the network
```
public_net:
    driver: macvlan
    driver_opts:
        parent: net2
    ipam:
        config:
            - subnet: 192.168.70.128/26
    name: oai-cn5g-public-net
```